# Welcome to Core Session 3!

## AGENDA

1. OpenAI Agents SDK  
  - Recap of Async Python and Pydantic
  - Agents with Tools and Structured Outputs 
  - 3 ways for Agents to Collaborate  
2. Use Case: Research Agent
  - Deep Research Prototype
  - Deep Research App
3. Going Deeper 
  - Coder Agents
  - Operator Agents
  - Digital Me revisited


## Approach

- Interactive
- More coding
- Fewer slides...
- Some more advanced stuff

# Lab 1: Foundations

## Topic 1: Async

In [1]:
# from functions to co-routines

def hello():
    print("Hey!")

# And now call it:


In [2]:
async def hello():
    print("Hey!")

# And now call it without await:

In [4]:
hello()

<coroutine object hello at 0x000001630F8E8400>

In [5]:
async def hello():
    print("Hey!")

# And now call it with await:

In [6]:
await hello()
await hello()
await hello()

Hey!
Hey!
Hey!


In [7]:
import asyncio

async def do_some_work(id):
    print(f"Starting work {id}")
    await asyncio.sleep(1)
    print(f"Completed work {id}")

In [8]:
await do_some_work(1)

Starting work 1
Completed work 1


In [9]:
coroutine = do_some_work(1)

In [10]:
await coroutine

Starting work 1
Completed work 1


In [11]:
await do_some_work(1)
await do_some_work(2)
await do_some_work(3)

Starting work 1
Completed work 1
Starting work 2
Completed work 2
Starting work 3
Completed work 3


In [12]:
# And now!!

await asyncio.gather(do_some_work(1), do_some_work(2), do_some_work(3))


Starting work 1
Starting work 2
Starting work 3
Completed work 1
Completed work 2
Completed work 3


[None, None, None]

In [13]:
# But - check this out:

import time

async def do_some_work(id):
    print(f"Starting work {id}")
    time.sleep(1)
    print(f"Completed work {id}")

In [14]:
await asyncio.gather(do_some_work(1), do_some_work(2), do_some_work(3))

Starting work 1
Completed work 1
Starting work 2
Completed work 2
Starting work 3
Completed work 3


[None, None, None]

## It looks like threading.. but it's not..

"Cooperative Multi-tasking"

Discuss the similarities and differences

When is it better to use multi-threading (or multi-processing) versus Async?

Why is it so common with LLM projects, and Agents in particular? Why not use multi-processing?

## Topic 2: Pydantic

Pydantic is a Data Validation and Schema framework.

Useful for describing the format of structured data (like json), in a way that allows us to be more rigorous. 

In [15]:
from pydantic import BaseModel, Field
from typing import Optional

In [16]:
# The old way

book_dict = {"title": "The Great Gatsby", "author": "F. Scott Fitzgerald"}
print(book_dict["title"])
print(book_dict["author"])



The Great Gatsby
F. Scott Fitzgerald


In [17]:
# The new way

class Book(BaseModel):
    title: str
    author: str

book = Book(title="The Great Gatsby", author="F. Scott Fitzgerald")
print(book.title)
print(book.author)
print(book)

The Great Gatsby
F. Scott Fitzgerald
title='The Great Gatsby' author='F. Scott Fitzgerald'


In [18]:
# Convenient shortcut, rather than specifying each field

book = Book(**book_dict)
book

Book(title='The Great Gatsby', author='F. Scott Fitzgerald')

In [19]:
# Newer way in latest Pydantic

book = Book.model_validate(book_dict)
book

Book(title='The Great Gatsby', author='F. Scott Fitzgerald')

### So structured data in Python could be represented in 3 ways:

1. As a JSON string
2. As a Python dict
3. As a Pydantic object

Let's explore all 3, and see how to convert between them

In [20]:
book_json = book.model_dump_json()

print(book_json)
print(book_dict)
print(book)

{"title":"The Great Gatsby","author":"F. Scott Fitzgerald"}
{'title': 'The Great Gatsby', 'author': 'F. Scott Fitzgerald'}
title='The Great Gatsby' author='F. Scott Fitzgerald'


In [21]:
this_is_json = '{"title": "The Great Gatsby", "author": "F. Scott Fitzgerald"}'
this_is_dict = {"title": "The Great Gatsby", "author": "F. Scott Fitzgerald"}
this_is_pydantic_object = Book(title="The Great Gatsby", author="F. Scott Fitzgerald")


In [24]:
book_json = book.model_dump_json(indent=4)
print(book_json)

{
    "title": "The Great Gatsby",
    "author": "F. Scott Fitzgerald"
}


In [23]:
# ERROR - create a pydantic object- cant parse just a string

book = Book(**book_json)

TypeError: __main__.Book() argument after ** must be a mapping, not str

In [25]:
book = Book.model_validate_json(book_json)
book

Book(title='The Great Gatsby', author='F. Scott Fitzgerald')

In [26]:
# To dict

book.model_dump()

{'title': 'The Great Gatsby', 'author': 'F. Scott Fitzgerald'}

In [27]:
# To json

book.model_dump_json()

'{"title":"The Great Gatsby","author":"F. Scott Fitzgerald"}'

## Pydantic Validation - Optional vs Default

This is confusing!

You can set default values and Optional types, per below. But this might not have the exact implications that you expect..

In [29]:
class BookCheck(BaseModel):
    title: str = "Untitled"
    author: Optional[str]


In [30]:
# Is this OK?

book_json = '{"title": "The Great Gatsby"}'
book = BookCheck.model_validate_json(book_json)

ValidationError: 1 validation error for BookCheck
author
  Field required [type=missing, input_value={'title': 'The Great Gatsby'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing

In [31]:
# Is this OK?

book_json = '{"author": null}'
book = BookCheck.model_validate_json(book_json)
book

BookCheck(title='Untitled', author=None)

## Pydantic Validation - new fields

Silently discards extra fields by default; can be strict, and can accept extras

In [32]:
book_special_json = {"title": "The Great Gatsby", "author": "F. Scott Fitzgerald", "pages": 180}
book_special_json

book_special = Book.model_validate(book_special_json)
book_special

Book(title='The Great Gatsby', author='F. Scott Fitzgerald')

In [33]:
book_special

Book(title='The Great Gatsby', author='F. Scott Fitzgerald')

In [34]:
class BookStrict(BaseModel):
    title: str
    author: str

    class Config:
        extra = "forbid"

book_strict = BookStrict.model_validate(book_special_json)
book_strict

ValidationError: 1 validation error for BookStrict
pages
  Extra inputs are not permitted [type=extra_forbidden, input_value=180, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/extra_forbidden

In [35]:
class BookFlexible(BaseModel):
    title: str
    author: str

    class Config:
        extra = "allow"

In [36]:
book_special = BookFlexible.model_validate(book_special_json)
book_special

BookFlexible(title='The Great Gatsby', author='F. Scott Fitzgerald', pages=180)

## Annotating fields

You can add descriptions to fields.

In [37]:
class BookExplained(BaseModel):
    title: str = Field(description="The title of the book")
    author: str = Field(description="The author of the book")


BookExplained.model_json_schema()

{'properties': {'title': {'description': 'The title of the book',
   'title': 'Title',
   'type': 'string'},
  'author': {'description': 'The author of the book',
   'title': 'Author',
   'type': 'string'}},
 'required': ['title', 'author'],
 'title': 'BookExplained',
 'type': 'object'}